# A360 Shear - Magnitude Distributions After Cuts

Contact author: Miranda Gorsuch

The goal here is to understand how the magnitude distribution in the Metadetection catalog changes as different cuts are applied. This is also partially to understand how sensitive the catalog is to S/N cuts, or if the other quality cuts take care of most of these issues.

For an example of a similar plot, see Figure 2 in [arXiv:1208.0605](https://arxiv.org/abs/1208.0605).

The 0.5 degree cut is applied prior to all cuts for consistency. This is also for easier comparison to the HSM cuts.

The cuts:
- **All detections**: all detections that have a finite *i*-band magnitude
- **Good detections**: flagged objects and duplicate objects removed
- **RS Cuts**: Red sequence galaxies are identified and removed
- **Masks**: Masks for bright objects, SFD dust map, and galactic cirrus are applied
- **WL Cuts**: General quality checks
- **Star-galaxy**: Size ratio cut to remove stars from the sample
- **Magnitude Cut**: the magnitude where SNR begins to remove objects
- **SNR Cut**: Objects with SNR < 10 are removed

Last working weekly: `w_2025_41`

Container Size: 16 GB (large)

## Read in data - All detections

In [ ]:
from lsst.daf.butler import Butler

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib

import astropy.units as u
from astropy.table import Table, join, vstack, hstack
from astropy.coordinates import SkyCoord

import gc
import healsparse as hsp

%matplotlib inline

# Position in degrees of the BCG for A360
ra_bcg = 37.865017
dec_bcg = 6.982205

REPO = '/sdf/data/rubin/repo/main/'
butler = Butler(REPO)
registry = butler.registry

In [ ]:
meas_type = 'gauss'

if meas_type == 'wmom':
    collection = 'u/mgorsuch/metadetect/a360_3_band/noise/20250827T213552Z' # with S/N fix and correct noise correlations
elif meas_type == 'gauss':
    collection = 'u/mgorsuch/metadetect/a360_3_band_gauss/noise/20250827T214428Z' # with noise realizations
elif meas_type == 'pgauss':
    collection = 'u/mgorsuch/metadetect/a360_3_band_pgauss/noise/20250827T215647Z' # with noise realization
else:
    print("Not an available measurement type")

# without noise realizations for testing
# if meas_type == 'wmom':
#     collection = 'u/mgorsuch/metadetect/a360_3_band/20250707T015251Z'
# elif meas_type == 'gauss':
#     collection = 'u/mgorsuch/metadetect/a360_3_band_gauss/20250812T015828Z'
# elif meas_type == 'pgauss':
#     collection = 'u/mgorsuch/metadetect/a360_3_band_gauss/20250814T154913Z' # I put this one in the wrong folder by mistake
# else:
#     print("Not an available measurement type")

In [ ]:
datasetRefs_shear = []
overlap_patches_10463 = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
tract_patch_list = [] # only used for plotting distribution of input images

for ref in butler.registry.queryDatasets('ShearObject', collections=collection):

    # the only parts of these tracts within 0.5 radius overlap with already included patches
    if ref.dataId['tract'] == 10704 or ref.dataId['tract'] == 10705:
        continue

    # these column of patches overlap with patches already in tract 10464
    if ref.dataId['tract'] == 10463 and ref.dataId['patch'] in overlap_patches_10463:
        continue
    
    datasetRefs_shear.append(butler.query_datasets('ShearObject', 
                                                     collections=collection,
                                                     skymap = 'lsst_cells_v1',
                                                     tract=ref.dataId['tract'],
                                                     patch=ref.dataId['patch']))

    tract_patch_list.append([ref.dataId['tract'], ref.dataId['patch']])

# combine the data from each patch into a single table
shear_table_list = []

for i, ref in enumerate(datasetRefs_shear):
    shear_data_patch = butler.get(ref[0])
    shear_table_patch = shear_data_patch.to_pandas()
    shear_table_list.append(shear_table_patch)

shear_table = pd.concat(shear_table_list)

# remove unused tables to clear up memory
del shear_table_list
gc.collect()

t1 = Table.from_pandas(shear_table)
t1[f'{meas_type}_band_mag_i'] = (t1[f'{meas_type}_band_flux_i']*u.nJy).to(u.ABmag)
shear_table = t1.to_pandas()

# Add columns for distance from BCG
c1 = SkyCoord(shear_table['ra'].values*u.deg, shear_table['dec'].values*u.deg)
c2 = SkyCoord(ra_bcg*u.deg, dec_bcg*u.deg)
sep = c1.separation(c2)

shear_table['deg_sep'] = sep.value

shear_table = shear_table[shear_table['deg_sep'] < 0.5]
all_detections = shear_table[shear_table['shear_type']=='ns'][f'{meas_type}_band_mag_i']
all_detections_frac = (len(all_detections) / len(shear_table[f'{meas_type}_band_mag_i'])) * 100

### Remove flagged objects

Can be skipped if reading in from a catalog.

In [ ]:
# meta_filter = shear_table[f'{meas_type}_flags']==False
# meta_filter &= shear_table['psfrec_flags']==False
# meta_filter &= shear_table[f'{meas_type}_psf_flags']==False
# meta_filter &= shear_table[f'{meas_type}_obj_flags']==False
# meta_filter &= shear_table[f'{meas_type}_T_flags']==False
# meta_filter &= shear_table[f'{meas_type}_band_flux_flags_r']==False
# meta_filter &= shear_table[f'{meas_type}_band_flux_flags_i']==False

# print("Prior to MD flags:")
# print("Number of rows prior to removing metadetect flags: ", len(shear_table))
# print("Number of rows in ns prior to removing metadetect flags: ", len(shear_table[shear_table['shear_type']=='ns']))
# print()

# shear_table = shear_table[meta_filter]

# print("After MD flags:")
# print("Number of rows after removing metadetect flags: ", len(shear_table))
# print("Number of rows in ns after removing metadetect flags: ", len(shear_table[shear_table['shear_type']=='ns']))

### Remove duplicate objects from patch and cell overlap

Can be skipped if reading in from a catalog.

In [ ]:
# # remove objects in outer ring of cells in each patch since patch overlap is two cells
# # TO-DO: exempt rows that don't overlap with other patches, e.g. patches on the edge of the field
# shear_table = shear_table[shear_table['cell_x']!=0]
# shear_table = shear_table[shear_table['cell_x']!=21]
# shear_table = shear_table[shear_table['cell_y']!=0]
# shear_table = shear_table[shear_table['cell_y']!=21]
# print("Number of rows after removing most duplicate cells: ", len(shear_table))
# print("Number of rows after removing most duplicate cells in ns: ", len(shear_table[shear_table['shear_type']=='ns']))

# # some additional tract/patch overlap appears to have a 4 cell overlap
# filt1 = shear_table['tract'] == 10464
# filt1 &= shear_table['patch_x'] == 9
# filt1 &= shear_table['cell_x'] == 20
# shear_table = shear_table[np.invert(filt1)]

# filt2 = shear_table['tract'] == 10463
# filt2 &= shear_table['patch_x'] == 1
# filt2 &= shear_table['cell_x'] == 1
# shear_table = shear_table[np.invert(filt2)]

# print()
# print("Number of rows after removing patch overlap areas: ", len(shear_table))
# print("Number of rows after removing patch overlap areas in ns: ", len(shear_table[shear_table['shear_type']=='ns']))

# # remove overlapping rows due to patch overlap    
# print("Number of rows prior to removing duplicates: ", len(shear_table))
# shear_table = shear_table.drop_duplicates(subset=['shear_type', 'ra', 'dec']) # each object will potentially have several sheared images
# print("Number of rows after removing duplicates: ", len(shear_table))
# print("Number of rows after removing duplicates in ns: ", len(shear_table[shear_table['shear_type']=='ns']))

# # concating the per-patch catalogs together results in non-unique indices
# shear_table.reset_index(inplace=True)

# # is there a better way to do this? Probably
# dup_cell_ndx = []

# for i, tract_patch in enumerate(tract_patch_list):
        
#     tract = tract_patch[0]
#     patch = tract_patch[1]

#     coadd = butler.get("deepCoaddCell", 
#                     tract=tract, 
#                     patch=patch, 
#                     band='i', 
#                     skymap='lsst_cells_v1', 
#                     collections=cell_collection)

#     shear_len_before = len(shear_table)

#     for cell_index in coadd.cells:
#         cell = coadd.cells[Index2D(x=cell_index.x, y=cell_index.y)]
#         cell_center = cell.inner.bbox.getCenter()

#         # find objects in this cell
#         filt_loc = shear_table['tract'] == tract
#         filt_loc &= shear_table['patch_y'] == (patch - (patch % 10)) // 10
#         filt_loc &= shear_table['patch_x'] == (patch % 10)
#         filt_loc &= shear_table['cell_x'] == cell_index.x
#         filt_loc &= shear_table['cell_y'] == cell_index.y

#         cell_objs = shear_table[filt_loc]

#         if len(cell_objs)==0:
#             continue
            
#         # identify those objects that are beyond the inner cell region 
#         # 75 pixels from the center in either dimension (inner cell length is 150 pixels)
#         outer_filt = cell_objs['col'] > (cell_center[0] + 75)
#         outer_filt |= cell_objs['col'] < (cell_center[0] - 75)
#         outer_filt |= cell_objs['row'] > (cell_center[1] + 75)
#         outer_filt |= cell_objs['row'] < (cell_center[1] - 75)

#         dup_cell_ndx.extend(cell_objs[outer_filt].index.to_list())

#     print(tract_patch)
#     del coadd
#     gc.collect()

# dup_cell_ndx = np.unique(dup_cell_ndx)
# shear_table.drop(dup_cell_ndx, inplace=True)

# print("Number of objects after duplicates removes: ", len(shear_table))
# print("Number of objects in ns after duplicates removes: ", len(shear_table[shear_table['shear_type']=='ns']))

### Load Catalog

In [ ]:
# read in catalog to skip other steps
if meas_type == 'wmom':
    shear_table = pd.read_pickle("./md-dup-wmom.pkl")
elif meas_type == 'gauss':
    shear_table = pd.read_pickle("./md-dup-gauss.pkl")
elif meas_type == 'pgauss':
    shear_table = pd.read_pickle("./md-dup-pgauss.pkl")
else:
    print("Not an available measurement type")

# without nosie realizations (for testing)
# read in catalog to skip other steps
# if meas_type == 'wmom':
#     shear_table = pd.read_pickle("./md-dup-wmom-no-noise.pkl")
# elif meas_type == 'gauss':
#     shear_table = pd.read_pickle("./md-dup-gauss-no-noise.pkl")
# elif meas_type == 'pgauss':
#     shear_table = pd.read_pickle("./md-dup-pgauss-no-noise.pkl")
# else:
#     print("Not an available measurement type")

shear_table = shear_table[shear_table['deg_sep'] < 0.5]

In [ ]:
good_detections = shear_table[shear_table['shear_type']=='ns'][f'{meas_type}_band_mag_i']
good_detections_frac = (len(good_detections) / len(shear_table[f'{meas_type}_band_mag_i'])) * 100

## RS Cuts

In [ ]:
mag_lim = 24
bright_lim = 18

# how many sigma to include in RS selection
sigma = 1

if meas_type == 'wmom':
    # slope, upper range, lower range
    rs_range = [(-0.09, 2.15, 1.85), # g-i
                (-0.03, 0.67, 0.47), # r-i
                (-0.07, 1.53, 1.37)] # g-r
elif meas_type == 'gauss':
    rs_range = [(-0.06, 2.05, 1.75),
                (-0.03, 0.63, 0.50),
                (-0.04, 1.43, 1.25)]
elif meas_type == 'pgauss':
    rs_range = [(-0.08, 2.05, 1.85),
                (-0.025, 0.62, 0.5),
                (-0.05, 1.45, 1.28)]
else:
    print("Not an available measurement type")
    
def RS_id(table, sigma, mag_lim, bright_lim, rs_range):

    # ranges
    gi_table_line_up = (table[f'{meas_type}_band_mag_r'] - 18) * (rs_range[0][0]) + rs_range[0][1]
    ri_table_line_up = (table[f'{meas_type}_band_mag_r'] - 18) * (rs_range[1][0]) + rs_range[1][1]
    gr_table_line_up = (table[f'{meas_type}_band_mag_r'] - 18) * (rs_range[2][0]) + rs_range[2][1]
    
    gi_table_line_down = (table[f'{meas_type}_band_mag_r'] - 18) * (rs_range[0][0]) + rs_range[0][2]
    ri_table_line_down = (table[f'{meas_type}_band_mag_r'] - 18) * (rs_range[1][0]) + rs_range[1][2]
    gr_table_line_down = (table[f'{meas_type}_band_mag_r'] - 18) * (rs_range[2][0]) + rs_range[2][2]
    
    gi_err = np.sqrt(table[f'{meas_type}_mag_g_err']**2 + table[f'{meas_type}_mag_i_err']**2)
    ri_err = np.sqrt(table[f'{meas_type}_mag_r_err']**2 + table[f'{meas_type}_mag_i_err']**2)
    gr_err = np.sqrt(table[f'{meas_type}_mag_g_err']**2 + table[f'{meas_type}_mag_r_err']**2)

    gi_err_up = table[f'{meas_type}_color_mag_g-i'] + sigma * gi_err
    gi_err_down = table[f'{meas_type}_color_mag_g-i'] - sigma * gi_err
    ri_err_up = table[f'{meas_type}_color_mag_r-i'] + sigma * ri_err
    ri_err_down = table[f'{meas_type}_color_mag_r-i'] - sigma * ri_err
    gr_err_up = table[f'{meas_type}_color_mag_g-r'] + sigma * gr_err
    gr_err_down = table[f'{meas_type}_color_mag_g-r'] - sigma * gr_err

    '''
    There are 4 cases that we want to consider for RS selection:
    - Case 1: RS data without errors is within RS selection
    - Case 2: Upper error bar end is within RS selection
    - Case 3: Lower error bar end is within RS selection
    - Case 4: Upper error bar is above RS selection
                AND lower error bar is below RS selection

    An object that falls within any of these cases should be selected.
    '''

    gi_redseq = np.logical_and.reduce((np.logical_or.reduce((
                                            np.logical_and.reduce((
                                                table[f'{meas_type}_color_mag_g-i'] < gi_table_line_up,
                                                table[f'{meas_type}_color_mag_g-i'] > gi_table_line_down,)),
                                            np.logical_and.reduce((
                                                gi_err_up < gi_table_line_up,
                                                gi_err_up > gi_table_line_down,)),
                                            np.logical_and.reduce((
                                                gi_err_down < gi_table_line_up,
                                                gi_err_down > gi_table_line_down,)),
                                            np.logical_and.reduce((
                                                gi_err_up > gi_table_line_up,
                                                gi_err_down < gi_table_line_down,)),
                                       )),
                                       table[f'{meas_type}_band_mag_r'] < mag_lim, 
                                       table[f'{meas_type}_band_mag_r'] > bright_lim))

    ri_redseq = np.logical_and.reduce((np.logical_or.reduce((
                                            np.logical_and.reduce((
                                                table[f'{meas_type}_color_mag_r-i'] < ri_table_line_up,
                                                table[f'{meas_type}_color_mag_r-i'] > ri_table_line_down,)),
                                            np.logical_and.reduce((
                                                ri_err_up < ri_table_line_up,
                                                ri_err_up > ri_table_line_down,)),
                                            np.logical_and.reduce((
                                                ri_err_down < ri_table_line_up,
                                                ri_err_down > ri_table_line_down,)),
                                            np.logical_and.reduce((
                                                ri_err_up > ri_table_line_up,
                                                ri_err_down < ri_table_line_down,)),
                                       )),
                                       table[f'{meas_type}_band_mag_r'] < mag_lim, 
                                       table[f'{meas_type}_band_mag_r'] > bright_lim))
    
    gr_redseq = np.logical_and.reduce((np.logical_or.reduce((
                                            np.logical_and.reduce((
                                                table[f'{meas_type}_color_mag_g-r'] < gr_table_line_up,
                                                table[f'{meas_type}_color_mag_g-r'] > gr_table_line_down,)),
                                            np.logical_and.reduce((
                                                gr_err_up < gr_table_line_up,
                                                gr_err_up > gr_table_line_down,)),
                                            np.logical_and.reduce((
                                                gr_err_down < gr_table_line_up,
                                                gr_err_down > gr_table_line_down,)),
                                            np.logical_and.reduce((
                                                gr_err_up > gr_table_line_up,
                                                gr_err_down < gr_table_line_down,)),
                                       )),
                                       table[f'{meas_type}_band_mag_r'] < mag_lim, 
                                       table[f'{meas_type}_band_mag_r'] > bright_lim))
    
    all_redseq = np.logical_and.reduce((gi_redseq, ri_redseq, gr_redseq))

    redeq_ind = [gi_redseq, ri_redseq, gr_redseq]

    lines = [[gi_table_line_up, gi_table_line_down],
             [ri_table_line_up, ri_table_line_down],
             [gr_table_line_up, gr_table_line_down]]

    return all_redseq, redeq_ind, lines

In [ ]:
all_redseq_prior, _, _ = RS_id(shear_table, sigma, mag_lim, bright_lim, rs_range)

shear_table_rs = shear_table[~all_redseq_prior]

rs_prior_cuts = shear_table_rs[shear_table_rs['shear_type']=='ns'][f'{meas_type}_band_mag_i']
rs_prior_frac = (len(rs_prior_cuts) / len(shear_table_rs[f'{meas_type}_band_mag_i'])) * 100

shear_table = shear_table_rs

## Mask Cuts

In [ ]:
mask_hsp = hsp.HealSparseMap.read('/home/b/bclevine/A360/A360_full_mask_hsp_128_131072.parquet')

In [ ]:
# mask = ~mask_hsp['full_mask'].get_values_pos(shear_table['ra'], shear_table['dec'], lonlat=True)
star_mask = ~mask_hsp['bo'].get_values_pos(shear_table['ra'], shear_table['dec'], lonlat=True) # bright objects
dust_mask = ~mask_hsp['sfd'].get_values_pos(shear_table['ra'], shear_table['dec'], lonlat=True) # masks from SFD dust map
hand_mask = ~mask_hsp['hand'].get_values_pos(shear_table['ra'], shear_table['dec'], lonlat=True)  # masks done by hand, e.g. galactic cirrus
exp_mask = ~mask_hsp['exp'].get_values_pos(shear_table['ra'], shear_table['dec'], lonlat=True) # edge / num exp mask

mask = star_mask & dust_mask & hand_mask & exp_mask

print("Number of rows in ns before mask applied: ", len(shear_table[shear_table['shear_type']=='ns']))
shear_table = shear_table[mask]
print("Number of rows in ns after mask applied : ", len(shear_table[shear_table['shear_type']=='ns']))

In [ ]:
mask_cuts = shear_table[shear_table['shear_type']=='ns'][f'{meas_type}_band_mag_i']
mask_cuts_frac = (len(mask_cuts) / len(shear_table[f'{meas_type}_band_mag_i'])) * 100

## WL Cuts

Primarily star-galaxy separtation and applying masks

In [ ]:
# store cut values in a dictionary
if meas_type == 'wmom':
    mag_i = 24.0
    t_ratio = 1.05
elif meas_type == 'gauss':
    mag_i = 23.5
    t_ratio = 0.2
elif meas_type == 'pgauss':
    mag_i = 23.75
    t_ratio = 0.2
else:
    print("Not an available measurement type")

cut_dict = {'s2n' : 10,
            'T' : 20,
            'mfrac' : 0.1,
            'color' : 5,
            'bright' : 20,
            'mag_i' : mag_i,
            't_ratio' : t_ratio}

In [ ]:
sigma = 0

final_cuts = shear_table[f'{meas_type}_T']<cut_dict['T']
final_cuts &= shear_table['mfrac']<cut_dict['mfrac']
final_cuts &= (shear_table[f'{meas_type}_color_mag_g-r']).abs()<cut_dict['color']
final_cuts &= (shear_table[f'{meas_type}_color_mag_r-i']).abs()<cut_dict['color']
final_cuts &= (shear_table[f'{meas_type}_color_mag_g-i']).abs()<cut_dict['color']
final_cuts &= shear_table[f'{meas_type}_band_mag_i']>cut_dict['bright']
# final_cuts &= shear_table['deg_sep'] < 0.5

shear_table_wl = shear_table[final_cuts]

In [ ]:
wl_cuts = shear_table_wl[shear_table_wl['shear_type']=='ns'][f'{meas_type}_band_mag_i']
wl_cuts_frac = (len(wl_cuts) / len(shear_table_wl[f'{meas_type}_band_mag_i'])) * 100

## Star-galaxy cut

In [ ]:
sg_filt = shear_table[f'{meas_type}_T_ratio']>cut_dict['t_ratio']
sg_filt &= (shear_table[f'{meas_type}_T_ratio'] - sigma*shear_table[f'{meas_type}_T_err'])>cut_dict['t_ratio']

shear_table_wl = shear_table_wl[sg_filt]

In [ ]:
sg_cuts = shear_table_wl[shear_table_wl['shear_type']=='ns'][f'{meas_type}_band_mag_i']
sg_cuts_frac = (len(sg_cuts) / len(shear_table_wl[f'{meas_type}_band_mag_i'])) * 100

## Get some cuts prior to magnitude cut

In [ ]:
shear_ns_pre_mag = shear_table_wl[shear_table_wl['shear_type']=='ns'].copy()

## Magnitude Cut

In [ ]:
mag_diff = cut_dict['mag_i']

shear_table_wl = shear_table_wl[shear_table_wl[f'{meas_type}_band_mag_i']<mag_diff]
mag_cut = shear_table_wl[shear_table_wl['shear_type']=='ns'][f'{meas_type}_band_mag_i']
mag_cut_frac = (len(mag_cut) / len(shear_table_wl[f'{meas_type}_band_mag_i'])) * 100

## SNR Cut 

In [ ]:
shear_table_wl = shear_table_wl[shear_table_wl[f'{meas_type}_s2n']>cut_dict['s2n']]
snr_cut = shear_table_wl[shear_table_wl['shear_type']=='ns'][f'{meas_type}_band_mag_i']
snr_cut_frac = (len(snr_cut) / len(shear_table_wl[f'{meas_type}_band_mag_i'])) * 100

shear_ns_pre_mag = shear_ns_pre_mag[shear_ns_pre_mag[f'{meas_type}_s2n']>cut_dict['s2n']]
snr_cut_pre_mag = shear_ns_pre_mag[f'{meas_type}_band_mag_i']

## Histogram

In [ ]:
# plot with RS as done in analysis
# bins = np.histogram(np.hstack((wl_cuts.dropna().values, mag_cut, snr_cut, rs_cuts)), bins=40)[1]
bins = np.histogram(np.hstack((mask_cuts.dropna().values, wl_cuts.dropna().values, sg_cuts, mag_cut, snr_cut)), bins=45)[1]

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,7))

ax[0].hist(all_detections, bins=bins, histtype='step', linewidth=1, label='All detections', color='blue')
ax[0].hist(good_detections, bins=bins, histtype='step', linewidth=1, label='Good detection', color='skyblue')
ax[0].hist(rs_prior_cuts, bins=bins, histtype='step', linewidth=1, label='RS Prior Cuts', color='red')
ax[0].hist(mask_cuts, bins=bins, histtype='step', linewidth=1, label='Masks applied', color='green')
ax[0].hist(wl_cuts, bins=bins, histtype='step', linewidth=1, label='WL Cuts', color='orange', linestyle=(0, (5, 10)))
ax[0].hist(sg_cuts, bins=bins, histtype='step', linewidth=1, label='Star-galaxy Cuts', color='purple')
ax[0].hist(mag_cut, bins=bins, histtype='step', linewidth=1, label=f'i_mag<{mag_diff}', color='black')
ax[0].hist(snr_cut, bins=bins, histtype='step', linewidth=1, label='SNR>10', color='cyan', linestyle=(0, (5, 10)), zorder=2.5)
# ax[0].hist(rs_cuts, bins=bins, histtype='step', linewidth=1, label='RS Cuts', color='red')

ax[0].set_title('Magnitude Cut Applied')
ax[0].set_ylabel('Counts', labelpad=15)
ax[0].set_xlabel('i-mag', labelpad=15)
ax[0].set_yscale('log')
ax[0].set_ylim(50, 40000)
ax[0].set_xlim(18, 26)
ax[0].legend()
# teal
ax[1].hist(all_detections, bins=bins, histtype='step', linewidth=1, label='All detections', color='blue')
ax[1].hist(good_detections, bins=bins, histtype='step', linewidth=1, label='Good detection', color='skyblue')
ax[1].hist(rs_prior_cuts, bins=bins, histtype='step', linewidth=1, label='RS Prior Cuts', color='red')
ax[1].hist(mask_cuts, bins=bins, histtype='step', linewidth=1, label='Masks applied', color='green')
ax[1].hist(wl_cuts, bins=bins, histtype='step', linewidth=1, label='WL Cuts', color='orange', linestyle=(0, (5, 10)))
ax[1].hist(sg_cuts, bins=bins, histtype='step', linewidth=1, label='Star-galaxy Cuts', color='purple')
ax[1].hist(snr_cut_pre_mag, bins=bins, histtype='step', linewidth=1, label='SNR>10', color='cyan', linestyle=(0, (5, 10)), zorder=2.5)
# ax[1].hist(rs_cut_pre_mag, bins=bins, histtype='step', linewidth=1, label='RS Cuts', color='red')
ax[1].vlines(mag_diff, 0, 40000, color='black', label=f'i_mag<{mag_diff}')

ax[1].set_title('No Magnitude Cut')
ax[1].set_xlabel('i-mag', labelpad=15)
ax[1].set_yscale('log')
ax[1].set_ylim(50, 40000)
ax[1].set_xlim(18, 26)

plt.suptitle('Magnitude Distributions', fontsize=14)
fig.savefig(f'image_outputs_{meas_type}/mag-cut-hist2.png')
# fig.savefig(f'image_outputs_{meas_type}_no_noise/mag-cut-hist2.png')

In [ ]:
print("Number of objects for each cut\n")

print(f"All detections: {len(all_detections)}, ({all_detections_frac}%)")
print(f"Good detections: {len(good_detections)}, ({good_detections_frac}%)")
print(f"RS Cut: {len(rs_prior_cuts)}, ({rs_prior_frac}%)")
print(f"Masks: {len(mask_cuts)}, ({mask_cuts_frac}%)")
print(f"WL Cuts: {len(wl_cuts)}, ({wl_cuts_frac}%)")
print(f"Star-Galaxy: {len(sg_cuts)}, ({sg_cuts_frac}%)")
print(f"i-mag < {mag_diff}: {len(mag_cut)}, ({mag_cut_frac}%)")
print(f"SNR > 10: {len(snr_cut)}, ({snr_cut_frac}%)")

The deviation between the SNR cut and `i_mag` cut begins for the three methods around `i-mag`:
- `wmom`: 24.5
- `gauss`: 23.5
- `pgauss`: 23.75